In [1]:
from netgen.geom2d import SplineGeometry
from ngsolve import *

from xfem import *
from xfem.mlset import *
from math import pi,e
from numpy import linspace
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import svds

DrawDC = MakeDiscontinuousDraw(Draw)

importing ngsxfem-2.1.2504


In [2]:
def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(2)

In [3]:
# physical parameters for linear elastic
mu  = 10
lam = 100

# parameters of DG method
order_u = 1
beta_u = 200

# parameter of ghost penalty
gamma_u = 10


In [4]:
# u_x = 4*(x-1)*(x+1)*(y-0.5)*(y+0.5)
# u_y = 8*(x-1)*(x+1)*(y-0.5)*(y+0.5)

u_x = x**2
u_y = y**2

In [5]:
# manufactured solution
exact_u = CF((u_x, u_y))

# strain tensor
epsilon_xx = u_x.Diff(x)
epsilon_yy = u_y.Diff(y) 
epsilon_xy = 0.5*(u_x.Diff(y) +  u_y.Diff(x))


# total stress tensor
sigma_xx = lam*(epsilon_xx + epsilon_yy) + 2*mu*epsilon_xx 
sigma_yy = lam*(epsilon_xx + epsilon_yy) + 2*mu*epsilon_yy 
sigma_xy = 2*mu*epsilon_xy


# 右端项 f_x, f_y
f_x = - (sigma_xx.Diff(x) + sigma_xy.Diff(y))
f_y = - (sigma_xy.Diff(x) + sigma_yy.Diff(y))

fe = CF((f_x, f_y))

uD = exact_u

In [6]:
# Construct background mesh
# Geometry and Mesh
h0 = 1/32
quad_mesh = False
geo = SplineGeometry()
geo.AddRectangle((-1, -0.5), (1, 0.5), bc="outer")
ngmesh = geo.GenerateMesh(maxh=h0, quad_dominated=quad_mesh)
mesh = Mesh(ngmesh)
# Draw(mesh)
# print(mesh.GetBoundaries())

In [7]:
circle1 = 1/9-(x-0.5)**2-y**2
circle2 = 1/9-(x+0.5)**2-y**2
level_sets = (circle1, circle2)
nr_ls = len(level_sets)
level_sets_p1 = tuple(GridFunction(H1(mesh,order=1)) for i in range(nr_ls))

for i, lset_p1 in enumerate(level_sets_p1):
    InterpolateToP1(level_sets[i],lset_p1)
    # DrawDC(lset_p1,-3.5,2.5,mesh)

In [17]:
omega = DomainTypeArray((NEG,NEG))
boundary = omega.Boundary()

mlci = MultiLevelsetCutInfo(mesh,level_sets_p1)
els_hasneg = mlci.GetElementsWithContribution(omega)
els_if = mlci.GetElementsWithContribution(boundary)
facets_gp = GetFacetsWithNeighborTypes(mesh,a=els_hasneg,b=els_if,use_and=True)
interior_facets = GetFacetsWithNeighborTypes(mesh,a=els_hasneg,b=els_hasneg,bnd_val_a=False,bnd_val_b=False,use_and=True)

Draw(BitArrayCF(els_hasneg),mesh)
# Draw(BitArrayCF(els_if),mesh)

els_if_single = {}
for i,dtt in enumerate(boundary):
    els_if_single[dtt] = mlci.GetElementsWithContribution(dtt)
    # Draw(BitArrayCF(els_if_single[dtt]),mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [9]:
# ba_surround_facets = GetElementsWithNeighborFacets(mesh,interior_facets)
# Draw(BitArrayCF(ba_surround_facets), mesh, "ba_surround")

In [10]:
# print(help(dCut))

In [11]:
Uhbase = VectorL2(mesh, order=order_u, dirichlet=[], dgjumps=True) # space for displacement
U = Compress(Uhbase, GetDofsOfElements(Uhbase, els_hasneg))
# freedofs = GetDofsOfElements(U,els_hasneg) & U.FreeDofs()
u,v = U.TnT()
h = specialcf.mesh_size
ne = specialcf.normal(mesh.dim) # normal vectors on faces
normals = omega.GetOuterNormals(level_sets_p1) # levelsets的外法向量
domega = dCut(level_sets_p1,omega,definedonelements=els_hasneg)
dk = dCut(level_sets_p1, omega, skeleton=True, definedonelements=interior_facets)
# dk = dx(skeleton=True,definedonelements=interior_facets)
dsc = {dtt:dCut(level_sets_p1,dtt,definedonelements=els_if_single[dtt]) for dtt in boundary}
dsbar = ds(skeleton=True)
dw = dFacetPatch(definedonelements=facets_gp)

In [12]:
strain_u = Sym(Grad(u))
strain_v = Sym(Grad(v))
mean_stress_u = 0.5*(Stress(Sym(Grad(u)))+Stress(Sym(Grad(u.Other()))))*ne
mean_stress_v = 0.5*(Stress(Sym(Grad(v)))+Stress(Sym(Grad(v.Other()))))*ne
jump_u = u - u.Other()
jump_v = v - v.Other()

jump_du = Grad(u)*ne - Grad(u.Other())*ne
jump_dv = Grad(v)*ne - Grad(v.Other())*ne

In [13]:
# stiffness matrix
Ah = BilinearForm(U)
Ah += 2*mu*InnerProduct(strain_u,strain_v)*domega + lam*div(u)*div(v)*domega
Ah += - (InnerProduct(mean_stress_u,jump_v) + InnerProduct(mean_stress_v,jump_u))*dk 
# boundary terms
Ah += - (InnerProduct(Stress(Sym(Grad(u)))*ne,v) + InnerProduct(Stress(Sym(Grad(v)))*ne,u))*dsbar # outer boundary
# levelset boundaries
for bnd, n in normals.items():
    Ah += -InnerProduct(Stress(Sym(Grad(u)))*n,v) * dsc[bnd]
    Ah += -InnerProduct(Stress(Sym(Grad(v)))*n,u) * dsc[bnd]
    
# ghost penalty terms
Ah += gamma_u / (h**2) * jump_u * jump_v * dw


# r.h.s
lh = LinearForm(U) 
lh += fe*v*domega - InnerProduct(uD,Stress(Sym(Grad(v)))*n)*dsbar 
for bnd, n in normals.items():
    lh += -InnerProduct(uD,Stress(Sym(Grad(v)))*n)*dsc[bnd] 


In [14]:
NitschType = 2
if NitschType == 1:
    Ah += beta_u/h*InnerProduct(jump_u,jump_v)*dk  # interior jump
    Ah += beta_u/h*InnerProduct(u,v)*dsbar 
    lh += beta_u/h*InnerProduct(uD,v)*dsbar
    for bnd, n in normals.items():
        Ah += beta_u / h * InnerProduct(u, v) * dsc[bnd] # nitsche term
        lh += beta_u/h*InnerProduct(uD,v)*dsc[bnd] 
elif NitschType == 2:
    Ah += beta_u/h*(2*mu*InnerProduct(jump_u,jump_v)+lam*InnerProduct(jump_u,ne)*InnerProduct(jump_v,ne))*dk  # interior jump
    Ah += beta_u/h*(2*mu*InnerProduct(u,v) + lam*InnerProduct(u,ne)*InnerProduct(v,ne))*dsbar  
    lh += beta_u/h*(2*mu*InnerProduct(uD,v) + lam*InnerProduct(uD,ne)*InnerProduct(v,ne))*dsbar
    for bnd, n in normals.items():
        Ah += beta_u/h*(2*mu*InnerProduct(u,v) + lam*InnerProduct(u,n)*InnerProduct(v,n))*dsc[bnd]   
        lh += beta_u/h*(2*mu*InnerProduct(uD,v) + lam*InnerProduct(uD,n)*InnerProduct(v,n))*dsc[bnd] 

In [15]:
Ah.Assemble()
lh.Assemble()
gfu = GridFunction(U)
gfu.vec.data = Ah.mat.Inverse() * lh.vec

catch in AssembleBilinearform 2: cut element boundary integrals not implemented for multi level sets


NgException: cut element boundary integrals not implemented for multi level setsin Assemble BilinearForm 'biform_from_py'


In [16]:
error_u = sqrt(Integrate((gfu - exact_u)**2 * domega.order(order_u), mesh))
print(error_u)

NameError: name 'gfu' is not defined

In [ ]:
mask = omega.Indicator(level_sets_p1)
# Draw(mask*gfu,mesh)
# Draw(mask*exact_u,mesh)
Draw(mask*(exact_u-gfu),mesh)

In [ ]:
deltau = CF((exact_u[0].Diff(x),exact_u[0].Diff(y),exact_u[1].Diff(x),exact_u[1].Diff(y)),dims=(2, 2)).Compile()

In [ ]:
grad_error_u = Grad(gfu)-deltau
error_u_H1 = sqrt(Integrate(InnerProduct(grad_error_u,grad_error_u)*domega, mesh))
print(error_u_H1)

In [ ]:
Draw(mask*Norm(grad_error_u),mesh)

In [ ]:
# gff = GridFunction(U)
# gff.Set(exact_u)
# grad_error_u = Grad(gfu - gff)
# error_u_H1 = sqrt(Integrate(InnerProduct(grad_error_u,grad_error_u)*domega, mesh))
# print(error_u_H1)

In [ ]:
# gff = GridFunction(Uhbase)
# gff.Interpolate(exact_u)
# err = gfu - gff
# strain_error = 0.5 * (Grad(err) + Grad(err).trans)
# # div_error = div(gfu) - div(gff)
# energy_error_sq = 0
# energy_error_sq += Integrate(InnerProduct(strain_error, strain_error)*domega,mesh)
# error_u_H1 = sqrt(energy_error_sq)
# print(error_u_H1)

In [ ]:
# gff = GridFunction(U)
# gff.Set(exact_u)
# grad_error_u = Grad(gfu - gff)
# error_u_H1 = sqrt(Integrate(InnerProduct(grad_error_u,grad_error_u)*  domega, mesh))
# print(error_u_H1)

In [ ]:
# DrawDC(lsetp1, gfu, (0,0), mesh, "uh", deformation=deformation)
# DrawDC(lsetp1, exact_u, (0,0), mesh, "uh", deformation=deformation)

In [ ]:
fesstress = MatrixValued(H1(mesh,order=order_u), symmetric=True)
gfstress = GridFunction(fesstress)

In [ ]:
# print(help(gfstress[0]))

In [ ]:
gfstress.Interpolate(Stress(Sym(deltau)))
# Draw(Norm(gfstress), mesh,deformation=deformation,min=0,max=500);
Draw(Norm(gfstress), mesh,deformation=deformation);

In [ ]:
gfstress.Interpolate(Stress(Sym(Grad(gff))))
Draw (Norm(gfstress), mesh,deformation=deformation);

In [ ]:
gfstress.Interpolate(Stress(Sym(Grad(gff-gfu))))
Draw (mask*Norm(gfstress[3]), mesh,deformation=deformation);

In [ ]:
gfstress.Interpolate(Stress(Sym(Grad(gfu)-deltau)))
Draw (mask*Norm(gfstress), mesh,min=0,max=0.1);
Draw (mask*Norm(gfstress), mesh,deformation=deformation);

In [ ]:
kappaminus = CutRatioGF(ci)
kappaminus_values = kappaminus.vec.FV().NumPy()
# positive_values = []
# positive_indices = []
# for v,ind in enumerate(kappaminus_values):
#     positive_values.append(v)
#     positive_indices.append(ind)
positive_values = [v for v in kappaminus_values if v > 0]
if positive_values:
    min_value_pythonic = min(positive_values)
    print(f"The smallest cut ratio is: {min_value_pythonic:.2e}")
else:
    print("There are no cut elements.")

In [ ]:
# positive_indices = np.array([v for v in range(len(kappaminus_values)) if kappaminus_values[v] > 0])
positive_indices = np.where(kappaminus_values > 0)[0]
# print(len(positive_values))
sorted_value_indices_in_positive_array = np.argsort(positive_values)
Ke = 10
smallest_K_indices_in_positive_array = sorted_value_indices_in_positive_array[:Ke]
smallest_K_original_indices = positive_indices[smallest_K_indices_in_positive_array]
min_K_values = kappaminus_values[smallest_K_original_indices]
print(f"最小的 {Ke} 个切割单元的比率: {min_K_values}")
print(f"它们在原始数组中的索引: {smallest_K_original_indices}")
print("-" * 30)

# visualize
smallest_K = BitArray(mesh.ne)
smallest_K.Clear()
for i in smallest_K_original_indices:
    smallest_K.Set(i)
Draw(BitArrayCF(smallest_K),mesh)

In [ ]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import svds

# 计算最大奇异值
rows,cols,vals = Ah.mat.COO()
A = sp.csr_matrix((vals,(rows,cols)))
condition_number = np.linalg.cond(A.todense())
# _, s_max,_ = svds(A, k=1, which='LM')   # largest magnitude

# # # 计算最小奇异值
# _, s_min,_ = svds(A, k=1, which='SM')   # smallest magnitude

# condition_number = s_max[0] / s_min[0]
print(f"{condition_number:.2e}")


In [ ]:
# Ah = BilinearForm(U)
# # Ae
# Ah += 2*mu*InnerProduct(strain_u,strain_v)*domega + lam*div(u)*div(v)*domega \
#         - (InnerProduct(mean_stress_u,jump_v) + InnerProduct(mean_stress_v,jump_u) - beta_u/h*InnerProduct(jump_u,jump_v))*dk \
#         - (InnerProduct(Stress(Sym(Grad(u)))*n,v) + InnerProduct(Stress(Sym(Grad(v)))*n,u) - beta_u/h*InnerProduct(u,v))*ds
# # order=1 i_s 
# Ah += gamma_u * h * ((Grad(u) - Grad(u.Other()))*ne) * ((Grad(v) - Grad(v.Other()))*ne) * dw
# # Ah += gamma_u * h * InnerProduct(Grad(u) - Grad(u.Other()),Grad(v) - Grad(v.Other())) * dw
# # Ah += gamma_u * h * jump_du * jump_dv * dw

# Ah.Assemble()

# # r.h.s
# lh = LinearForm(U) 
# lh += fe*v*domega - InnerProduct(uD,Stress(Sym(Grad(v)))*n)*ds + beta_u/h*uD*v*ds
# lh.Assemble()

# gfu = GridFunction(U)
# gfu.vec.data = Ah.mat.Inverse() * lh.vec